# Лабрраторная работа 3
## Акулич Кирилл 853504 (2 вариант)

$$ f(x,y) = \frac{1}{2}*sin(x+y)  \forall x, y: 0 \le x, y \le \frac{\pi}{2}$$

$$ 
    f(x) = \int_0^\frac{\pi}{2} \frac{1}{2}*sin(x+y) dy 
    f(x) = \frac{1}{2}*(sin(x) +cos(x)) = \frac{\sqrt{2}}{2}*cos(x-\frac{\pi}{4}) 
$$

Функцию распределения СВ х:
$$
    F(x) = \int_0^x f(x) dx = \int_0^x \frac{\sqrt{2}}{2}*cos(x-\frac{\pi}{4}) dx = \frac{\sqrt{2}}{2} * (sin(x-\pi/4) - \frac{\sqrt{2}}{2})
$$

Найдем функцию, обратную к данной
$$
    x = arcsin(\frac{2}{\sqrt{2}}*R - \frac{1}{\sqrt{2}}) + \pi/4
$$

Найдем условную функцию для y
$$
    
$$